### **Inicio de sesión en el sitio web - Construcción de la spider**

Vamos a hacer uso del sitioweb:

https://quotes.toscrape.com/login

Quiero recordarte que sólo hago este vídeo en concreto con fines educativos. No recomendaré scrapear sitios web que requieran loguearse. Esto no es una buena práctica, porque si el sitio web no quiere ser scrapeado, los propietarios no sólo banearán tu IP, sino también tu cuenta. Así que piensa dos veces antes de hacer esto, incluso si tus intenciones son buenas.

Comenzamos inspeccionando el sitioweb:

<center><img src="https://i.postimg.cc/6p19n4Gd/ws-205.png"></center>

A continuación, seleccionemos la pestaña "**`Red`**" y, después, hacemos clic en el botón "**`Todo`**" y marcamos la casilla "**`Conservar el registro`**". Esto nos ayudará a almacenar la solicitud (request) que enviamos cuando iniciamos sesión. Vamos a escribir el username y el password (no es necesario registrarse) y nos logeamos (el icono de prohibido nos sirve para limpiar la pantalla de inspeccion al momento de escribir nuestro usuario y contraseña):

<center><img src="https://i.postimg.cc/KYkmNcnC/ws-206.png"></center>

En la pestaña "**`Encabezados`**", podemos ver que la **`Request URL`** es **`quotes.toscrape.com/login`**. El **`Request Method`** es **`POST`**. Y el **`Status code`** es **`302`**, que representa la redirección de la URL. Antes de enviar el formulario, estábamos en /login. Después de enviarlo, fuimos redirigidos a la homepage. Esto es lo que representa el código de Estado 302.

<center><img src="https://i.postimg.cc/bJ1876VT/ws-207.png"></center>

En la pestaña **`Carga útil`** podemos ver un token que tiene un valor que es dinámico, es decir, cada vez se generará un valor nuevo. También nos muestra nuestro **`username`** y **`password`**.

<center><img src="https://i.postimg.cc/hGkc3Fsv/ws-208.png"></center>

Vamos a crear una nueva spider llamada **`quotes_login`**:

<center><img src="https://i.postimg.cc/nr9JKpN0/ws-209.png"></center>
<center><img src="https://i.postimg.cc/JtwWPSLm/ws-210.png"></center>

Cuando inspeccionamos el homepage del sitio necesitamos encontrar el código que almacene el token:

<center><img src="https://i.postimg.cc/9QhDfnqv/ws-211.png"></center>

Al momento de escribir el username y password y pulsar el botón de **`Login`**, si nos redirige correctamente a una página con contenido nos debiera aparecer un código que posea la palabra **`Logout`**:

<center><img src="https://i.postimg.cc/tCP1pj73/ws-212.png"></center>

El código que utilizaremos en el archivo **`quotes.py`**:

In [ ]:
import scrapy
from scrapy import FormRequest

class QuotesLoginSpider(scrapy.Spider):
    name = 'quotes_login'
    allowed_domains = ['quotes.toscrape.com']
    start_urls = ['https://quotes.toscrape.com/login']

    # Procesando el csrf_token, el username y la password
    def parse(self, response):
        csrf_token = response.xpath("//input[@name='csrf_token']/@value").get()
    # enviando FormRequest (FormRequest extiende el Request base con funcionalidad para tratar con formularios HTML)
    # FormRequest.from_response() simula el inicio de sesión de un usuario
        yield FormRequest.from_response(
            response,
            formxpath='//form',
            formdata={
                'csrf_token': csrf_token,
                'username': 'admin',
                'password': 'admin'
            },
            callback=self.after_login
        )
    # aquí definimos la función after_login que usamos en callback
    def after_login(self, response):
        # Si hay un texto "logout" en la página, imprime "Successfully logged in!"
        if response.xpath("//a[@href='/logout']/text()").get():
            print('Successfully logged in!')

Si ejecutamos por consola:

<center><img src="https://i.postimg.cc/CMRBrDg6/ws-213.png"></center>
<center><img src="https://i.postimg.cc/T2Byzw22/ws-214.png"></center>